In [11]:
import os
import nltk
import numpy as np
import copy
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
nltk.download('punkt')

documents = {}
labels = {}

# open main data
for file in os.listdir("/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training"):
    if file.endswith(".abstr"):
        content = open(("%s/%s" % ('/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training', file)), "r").read()
        documents[file.split('.')[0]] = content.split('. ')
        
# open labels        
for file in os.listdir("/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training"):
    if file.endswith(".uncontr"):
        content = open(("%s/%s" % ('/home/valeriya/Desktop/UMD/Computational_linguistic/Project/Hulth2003/Hulth2003/Training', file)), "r").read()
        labels[file.split('.')[0]] = content.split("; ")       

[nltk_data] Downloading package punkt to /home/valeriya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# tokenize document
tokenized_documents = {}
for num, ctt in documents.items():
    tokenized_documents[num] = []
    for sentence in ctt:
        words = nltk.word_tokenize(sentence.lower())
        #words=[word.lower() for word in words if word.isalpha()]
        tokenized_documents[num].append(words)

# tokenize labels
tokenized_labels = {}
for num, ctt in labels.items():
    tokenized_labels[num] = []
    for sentence in ctt:
        words = nltk.word_tokenize(sentence.lower())
        #words=[word.lower() for word in words if word.isalpha()]
        tokenized_labels[num].append(words)        

In [4]:
#PROBLEM 
# For document 1346 keyphrase should be "growing time adaptive self-organizing map", but word self-organized is lost
# Fixed

In [5]:
tokenized_labels['1346']

[['automatic', 'multilevel', 'thresholding'],
 ['image', 'segmentation'],
 ['growing', 'time', 'adaptive', 'self-organizing', 'map'],
 ['growing', 'tasom'],
 ['gtasom'],
 ['peak', 'finding', 'process']]

In [6]:
# function find index of element in list of lists
def index(lab, target):
    for i,phr in enumerate(lab):
        for j, w in enumerate(phr):
            if w == target:
                return (j)
    return (None, None)

# create dictionary of labels associated to words
class_labels = copy.deepcopy(tokenized_documents)
for document in tokenized_documents:
    text = tokenized_documents[document]
    lab = tokenized_labels[document]
    lab_flattened = [val for sublist in lab for val in sublist]
    for i, sentence in enumerate(text): 
        for j, word in enumerate(sentence): 
            is_keyphrase = word in lab_flattened
            if is_keyphrase:
                if index(tokenized_labels[document], word) == 0:
                    class_labels[document][i][j] = 1
                else:
                    class_labels[document][i][j] = 2
            else:
                class_labels[document][i][j] = 0

# GLOVE embeddings

In [15]:
import numpy as np
embeddings = dict()
embed_size = 100
f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

In [14]:
class biLSTM(nn.Module):
    def __init__(self):
        super(biLSTM, self).__init__()
        
        self.lstm = nn.LSTM(input_size, hidden_size = 150, num_layers = 1, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size = 150, 3) 
        
    def init_hidden(self,):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, 32, 150))
        
    def forward(self, X):
        lstm_out, self.hidden = self.lstm(X)      
        out = self.fc(self.hidden)
        
        return out

In [25]:
def SentToTensor(sentence):
    # returns tensor of size (number of words x 1 x length of embeddings)
    tensor = torch.zeros(len(sentence), 1, embed_size)
    for i, word in enumerate(sentence):
        tensor[i][0][:] = torch.tensor(embeddings[word])
    
    return(tensor)
        
print(sentToTensor(['the', 'a', 'king']).size())

def gen_batch(data, labels, size):
    X = torch.zeros(len(sentence), 1, embed_size)
    for i, (sample, label) in enumerate(list_of_tensors[1:]):
        X = torch.cat((X, sample), dim=0)
        y = torch.cat((y, label), dim=0)
    return X, y

torch.Size([3, 1, 100])


In [23]:


features = 1
seq_len = 10
hidden_size = 128
batch_size = 32

class BiLSTMNet(nn.Module):
    def __init__(self):
        super(BiLSTMNet, self).__init__()
        self.rnn1 = nn.GRU(input_size=features,
                            hidden_size=hidden_size,
                            num_layers=1)
        self.dense1 = nn.Linear(hidden_size, 1)

    def forward(self, x, hidden):
        x, hidden = self.rnn1(x, hidden)
        x = x.select(1, seq_len-1).contiguous()
        x = x.view(-1, hidden_size)
        x = self.dense1(x)
        return x, hidden

    def init_hidden(self):
        weight = next(self.parameters()).data
        return Variable(weight.new(1, batch_size, hidden_size).zero_())

model = Net()
model.cuda()
hidden = model.init_hidden()

X_train_1 = X_train[0:batch_size].reshape(seq_len,batch_size,features)
y_train_1 = y_train[0:batch_size]
model.zero_grad()
T = torch.Tensor
X_train_1, y_train_1 = T(X_train_1), T(y_train_1)
X_train_1, y_train_1 = Variable(X_train_1).cuda(), Variable(y_train_1).cuda()

output, hidden = model(X_train_1, Variable(hidden.data))

NameError: name 'Net' is not defined